In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
import xgboost

In [40]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Assuming df is your DataFrame and 'Feature17' has missing values
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df_train = pd.read_csv('../data/iith_foml_2023_train.csv')
# Fit the imputer model on the dataset to learn the data patterns
df_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
imputer.fit(df_train)

# Transform the dataset to replace missing values
df_imputed = imputer.transform(df_train)

# Convert back to a DataFrame
df_imputed = pd.DataFrame(df_imputed, columns=df_train.columns)

In [41]:
X_train = df_imputed.iloc[:, :-1]
y_train = df_imputed.iloc[:, -1]

In [42]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2)

print("Shape of train_x:", train_x.shape)
print("Shape of val_x:", val_x.shape)
print("Shape of train_y:", train_y.shape)
print("Shape of val_y:", val_y.shape)


Shape of train_x: (795, 22)
Shape of val_x: (199, 22)
Shape of train_y: (795,)
Shape of val_y: (199,)


In [43]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize XGBoost model
xgb_model = XGBClassifier(random_state=42, n_estimators=200)

le = LabelEncoder()
train_y = le.fit_transform(train_y)

# Train the model on the training data
xgb_model.fit(train_x, np.asarray(train_y, dtype=int))

# Make predictions on the validation data
xgb_predictions = xgb_model.predict(val_x)

# Calculate accuracy and macro F1 score
xgb_accuracy = accuracy_score(val_y, xgb_predictions)
xgb_macro_f1 = f1_score(val_y, xgb_predictions, average='macro', zero_division=0.0)

# Display results
print("XGBoost Accuracy:", xgb_accuracy)
print("XGBoost Macro F1 Score:", xgb_macro_f1)

# Classification report for more details
xgb_classification_rep = classification_report(val_y, xgb_predictions, zero_division=0.0)
print("\nXGBoost Classification Report:\n", xgb_classification_rep)


XGBoost Accuracy: 0.9095477386934674
XGBoost Macro F1 Score: 0.43913227513227515

XGBoost Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98        54
         1.0       0.91      0.97      0.94        97
         2.0       0.71      0.81      0.76        21
         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1
         5.0       1.00      0.56      0.71         9
         6.0       1.00      1.00      1.00        12
         8.0       0.00      0.00      0.00         2
         9.0       0.00      0.00      0.00         2
        12.0       0.00      0.00      0.00         0

    accuracy                           0.91       199
   macro avg       0.46      0.43      0.44       199
weighted avg       0.89      0.91      0.90       199

